In [48]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
#data = [open, high, low, close, volume USD, EMA difference], most recent entries first
data_newest = np.loadtxt(open("data/Bitstamp_Minutely.csv", "rb"), delimiter=",", skiprows=2, usecols=(3,4,5,6,8))

#oldest entries first data
X = data_newest[::-1]

def get_EMA(x, short=9, long=26, column=0):
    EMA = np.zeros((len(x),3))     #Returns EMA matrix with entires: [short period EMA, long period EMA, difference]
    
    #** REFER TO ABOVE MARKDOWN CELL
    EMA[long-1][0] = np.sum(x[long-short:long])/short   #These are SMAs used for the intial EMA calculation
    EMA[long-1][1] = np.sum(x[0:long])/long             #These are SMAs used for the intial EMA calculation

    #calculate EMA:
    w0 = 2/(1+short)   #short period weights
    w1 = 2/(1+long)    #long period weights
    for i in range(long, len(x)):
        EMA[i][0] = x[i]*w0 + EMA[i-1][0]*(1-w0)    #short term EMA 
        EMA[i][1] = x[i]*w1 + EMA[i-1][1]*(1-w1)    #long term EMA 
        EMA[i][2] = EMA[i][0]-EMA[i][1]
    return EMA

def attach_column(M, c):
    if M.shape[0] != len(c):
        print("Error: Cannot attach column to matrix with different sizes")
        return
    M = np.insert(M, M.shape[1], c, axis=1)
    return M

EMA = get_EMA(X[:,0])
X = attach_column(X, EMA[:,2])
X = X[27:]
y = np.zeros(X.shape[0])
for i in range(len(X)-5):
    y[i] = np.sign(np.average(X[i:i+5,0])-X[i][0])

In [49]:
X = X[:-5]
y = y[:-5]

In [50]:
dataset = attach_column(X, y)
np.save('data/labeled_data.npy', dataset)

In [52]:
y

array([-1., -1.,  1., ...,  1.,  1.,  1.])

In [54]:
data = np.load('data/labeled_data.npy')
scaler = MinMaxScaler()
        
labels = data[:, -1]
data = data[:, :-1]

data = scaler.fit_transform(data)

seq_len = 60
full_len = len(data) - seq_len
n_features = 6
X_sequenced = np.zeros((full_len, seq_len, n_features))
for i in range(full_len):
    X_sequenced[i] = data[i:i + seq_len]
labels = labels[seq_len:]
print(labels[-10:])

[-1. -1. -1. -1.  1.  1.  1.  1.  1.  1.]


In [55]:
labels[labels == -1] = 0

In [57]:
print(labels[-10:])

[0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]


In [20]:
import pandas as pd
import matplotlib.pyplot as plt
columns = ['Memory cells', 'Layers', 'Batch size', 'Sequence length', 'Accuracy']
hparams = [
    [1, 1, 64, 60, 0.527],
    [3, 2, 64, 60, 0.505],
    [5, 2, 256, 10, 0.500],
    [5, 2, 256, 120, 0.499],
    [5, 2, 1024, 60, 0.534],
    [10, 2, 64, 60, 0.504],
]
df = pd.DataFrame(hparams, columns=columns)
df.index.name='Model'


df.style.set_table_attributes("style='display:inline'").set_caption('LSTM models')
# df.head()

# fig, ax = plt.subplots(figsize=(16, 8))

# ax.table(cellText = df.values,
#           rowLabels = df.index,
#           colLabels = df.columns,
#           loc = "center"
#          )
# # ax.set_title("Top 10 Fields of Research by Aggregated Funding Amount")

# ax.axis("off")
# plt.show()

,Memory cells,Layers,Batch size,Sequence length,Accuracy
Model,,,,,
0,1,1,64,60,0.527000
1,3,2,64,60,0.505000
2,5,2,256,10,0.500000
3,5,2,256,120,0.499000
4,5,2,1024,60,0.534000
5,10,2,64,60,0.504000
